In [18]:
import cv2
import numpy as np

import torch
import torchvision.transforms as transforms
import gesture_detector.py 
a = gesture_detector.Network()

There are 87000 total train images.
There are 28 total test images.
There are 69600 total train images.
There are 17400 total dog validation images.
There are 28 total test images.
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 48, 48]             100
         MaxPool2d-2           [-1, 10, 24, 24]               0
            Conv2d-3           [-1, 20, 22, 22]           1,820
         MaxPool2d-4           [-1, 20, 11, 11]               0
            Conv2d-5             [-1, 30, 9, 9]           5,430
         Dropout2d-6             [-1, 30, 9, 9]               0
            Linear-7                  [-1, 270]         656,370
            Linear-8                   [-1, 29]           7,859
        LogSoftmax-9                   [-1, 29]               0
Total params: 671,579
Trainable params: 671,579
Non-trainable params: 0
------------------------------------------

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [17]:
model = Network()
model.load_state_dict(torch.load('saved_model.pt'))
model.eval()

NameError: name 'Network' is not defined

In [ ]:
gestures = {
    0:'A',
    1:'B',
    2:'C',
    3:'D',
    4:'E',
    5:'F',
    6:'G',
    7:'H',
    8:'I',
    9:'J',
    10:'K',
    11:'L',
    12:'M',
    13:'N',
    14:'O',
    15:'P',
    16:'Q',
    17:'R',
    18:'S',
    19:'T',
    20:'U',
    21:'V',
    22:'W',
    23:'X',
    24:'Y',
    25:'Z',
    26:'del',
    27:'nothing',
    28:'space'
    
}

In [ ]:
def predict(img_path):
    # load the image and return the predicted breed
    img = Image.open(img_path)
#     img = Image.open(img_path).convert('L')
    transformations = transforms.Compose([transforms.Grayscale(num_output_channels=1),
                                            transforms.Resize(size=50),
                                         transforms.ToTensor(),
                                         transforms.Normalize([0.5],[0.5])])
    image_tensor = transformations(img)[:3,:,:].unsqueeze(0)

    # move model inputs to cuda, if GPU available
    if use_cuda:
        image_tensor = image_tensor.cuda()

    # get sample outputs
    output = model(image_tensor)
    # convert output probabilities to predicted class
    _, preds_tensor = torch.max(output, 1)

    pred = np.squeeze(preds_tensor.numpy()[0]) if not use_cuda else np.squeeze(preds_tensor.cpu().numpy()[0])

    return gestures[pred]

In [ ]:
vc = cv2.VideoCapture(0)
rval, frame = vc.read()
old_text = ''
pred_text = ''
count_frames = 0
total_str = ''
flag = False

while True:
    
    if frame is not None: 
        
        frame = cv2.flip(frame, 1)
        frame = cv2.resize( frame, (400,400) )
        
        cv2.rectangle(frame, (300,300), (100,100), (0,255,0), 2)
        
        crop_img = frame[100:300, 100:300]
        grey = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
        
        thresh = cv2.threshold(grey,210,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)[1]
      
        
        blackboard = np.zeros(frame.shape, dtype=np.uint8)
        cv2.putText(blackboard, "Predicted text - ", (30, 40), cv2.FONT_HERSHEY_TRIPLEX, 1, (255, 255, 0))
        if count_frames > 20 and pred_text != "":
            total_str += pred_text
            count_frames = 0
            
        if flag == True:
            old_text = pred_text
            pred_text = predict(thresh)
        
            if old_text == pred_text:
                count_frames += 1
            else:
                count_frames = 0
            cv2.putText(blackboard, total_str, (30, 80), cv2.FONT_HERSHEY_TRIPLEX, 1, (255, 255, 127))
        res = np.hstack((frame, blackboard))
        
        cv2.imshow("image", res)
        cv2.imshow("hand", thresh)
        
    rval, frame = vc.read()
    keypress = cv2.waitKey(1)
    if keypress == ord('c'):
        flag = True
    if keypress == ord('q'):
        break

vc.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

vc.release()